## Init
To start have to run pip install for these packages. Once you run the next block you'll have to comment it out and run the rest of the document. Unfortunately, jupyter won't be able to find the command until you restart it.

In [1]:
# !pip install google-play-scraper
# This is for the sentiment analyzer. Since I used tensorflow, I have already installed it.
# !pip install transformers



In [2]:
# # https://medium.com/analytics-vidhya/google-play-store-apps-reviews-scraping-and-text-analytics-sentiment-analysis-5303294fffa7

# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# from google_play_scraper import app, Sort, reviews_all, permissions

## Downloading reviews from playstore


In [ ]:
def getReviewDf(appId):
    user_reviews = reviews_all(
        appId,
        sleep_milliseconds=0,
        lang='en',
        country='us',
        sort=Sort.NEWEST
    )
    # convert to pandas pd
    df_reviews = pd.DataFrame( np.array(user_reviews), columns=['review'])
    df_reviews = df_reviews.join(pd.DataFrame(df_reviews.pop('review').tolist()))
    return df_reviews   

def getReviewsFrom(app_dict):
    for app in app_dict.keys():
        appid= app_dict[app]
        user_reviews = reviews_all(
            appid,
            sleep_milliseconds=0,
            lang='en',
            country='us',
            sort=Sort.NEWEST
        )
        # convert to pandas pd
        df_reviews = pd.DataFrame( np.array(user_reviews), columns=['review'])
        df_reviews = df_reviews.join(pd.DataFrame(df_reviews.pop('review').tolist()))
        df_reviews.to_pickle(app)
        return df_reviews
    
    

In [ ]:
appsList = ['com.ChillyRoom.DungeonShooter',
 'com.bandainamcoent.digimon_rearise_ww',
 'com.sonypicturestelevision.WheelPuzzlePop',
 'com.istomgames.engine',
 'com.square_enix.android_googleplay.dq5',
 'jp.co.ponos.battlecatstw',
 'com.bandainamcoent.oprvww',
 'com.tgc.sky.android',
 'com.square_enix.android_googleplay.FFCCREww',
 'com.square_enix.android_googleplay.subarashikikonosekai_solo',
 'com.alpha.mpsen.android',
 'com.superplanet.swordmaster',
 'com.us.danmemo',
 'com.quesera.kalpa',
 'com.eyougame.xjhx.en',
 'com.andromedagames.noblesseglobal',
 'com.klab.utapri.shininglive.global']



import json
import os.path
from os import path

from google_play_scraper import app, Sort, reviews_all, permissions

for app_id in appsList:
    
    if path.exists('tempdfs/' + app_id):
        continue
    
    appData = None
    try:
        # fetch the review data
        temp_df = getReviewDf(app_id)

        if temp_df is None:
            continue
            
        # fetch the app information
        appData = app(app_id, lang='en', country='us')
    except Exception as e:
        print(e)
        print(app_id)
    
    if appData is None:
        continue
    
    title = appData['title']
#     display(title)
    
    temp_df['appId'] = appData['appId']
    temp_df['updated'] = appData['updated']
    temp_df['released'] = appData['released']
#     reviews_df['appId'] = reviews_df['appId'].apply(lambda x: appData['appId'] if title in x else x)
#     reviews_df['updated'] = reviews_df['updated'].apply(lambda x: appData['updated'] if title in x else x)
#     reviews_df['released'] = reviews_df['released'].apply(lambda x: appData['released'] if title in x else x)

    # we can clean up their timestamps while we are downloading
    
    from datetime import date,datetime
    temp_df['updated'] = temp_df['updated'].apply(lambda x: datetime.fromtimestamp(x))
    
    #   # let's put some review dates here
    # temp_df['review_posted_year'] = temp_df['at'].dt.year
    # temp_df['review_posted_month'] = temp_df['at'].dt.month
    # temp_df['review_posted_day'] = temp_df['at'].dt.day
    # temp_df['review_posted_dayofweek'] = temp_df['at'].dt.dayofweek
    # temp_df['review_posted_yearmonth'] = str(temp_df['at'].dt.year)+str(temp_df['at'].dt.month)

    # temp_df['app_created_year'] = temp_df['released'].dt.year
    # temp_df['app_lastupdated_year'] = temp_df['updated'].dt.year

    # # to help with analysis we can base the reviews on year 0, year 1...etc
    # temp_df['review_posted_year_norm'] = temp_df['review_posted_year'] - temp_df['app_created_year'] 
    # temp_df['review_posted_year_norm_months'] = temp_df['review_posted_year_norm'] +'-' + temp_df['review_posted_month'].str
  
    
    temp_df.to_pickle('tempdfs/'+app_id)  
    
#     display(appData['appId'])
#     display(appData['updated'])
#     display(appData['released'])
    
#  'released', 'updated','appId','title'

## Load sentiment model

In [ ]:
# from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer,TFAutoModel, TFAutoModelWithLMHead
import numpy as np
from scipy.special import softmax
import csv
import urllib.request

# # this is just a hack I tried to do to set the config directory.
# # in the end I downloaded all of the files locally and didn't need this
# %env SENTENCE_TRANSFORMERS_HOME=./.config
# %env TRANSFORMERS_CACHE=./transformers


MODEL = '/tf/school/UofM_School/Capstone/tweeteval_new/roberta-base-rt-sentiment'#tf_model.h5'

tokenizer = AutoTokenizer.from_pretrained(MODEL, local_files_only=True)#'tf_model.h5')
# tokenizer = AutoTokenizer.from_pretrained('./local_model_directory/')
model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)
labels = ['negative','neutral','positive']

## Load from directory and perform sentiment analysis

In [ ]:
## Load from directory and perform sentiment analysis

In [ ]:
def encode_text(text):
#     if text is None or text == '':
#         return ''
#     text = preprocess(text)
#     if text is None or text =='':
#         return
    encoded_input = tokenizer(text, return_tensors='tf', padding=True, truncation=True, max_length=512)
    return encoded_input
    
def GetReviewSentiment(encoded_input):
    output = model(encoded_input)
#     display(output)
    scores = output[0][0].numpy()
    scores = softmax(scores)

    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    return labels[ranking[0]]

In [ ]:
mypath = 'tempdfs/'


# get all files in the directory. Each file name should be the app-id that we used earlier
from os import listdir, path
from os.path import isfile, join
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [ ]:
def getSentiment(text):
        encoded_data = encode_text(list(text))
        output = model(encoded_data)
        scores = output[0].numpy()
        scores = softmax(scores, axis=1)
        ranking = np.argsort(scores,axis=1)
        ranking = np.flip(ranking, axis=1)
        sentiment = [labels[x[0]] for x in ranking]
        return sentiment

def batch_process_df(df, batch_size):
    
    
    timeEstimates = {}
    timeEstimates['count'] = 0
    timeEstimates['value'] = 0
    
    import time

#     start = time.time()
#     print("hello")
#     end = time.time()
#     print(end - start)

    numBatches = int(len(df)/batch_size)

    df['content'] = df['content'].apply(lambda x: 'NA' if x == None or x == 'None' else x)
    df['sentiment'] = df['content']
    
    final_df = None
    for i in range(0,numBatches):
        
        # show estimated time every 20 batches
        
        # we only want to show when the minute from last shown changes.
        remainingTime = int(round((timeEstimates['value'] * (numBatches-i))/60, 0))
        
#         changedMinute = int(round(timeEstimates['value']/60, 0)) != remainingTime
        if(timeEstimates['count'] > 0 and timeEstimates['count'] % 100 == 0):
            print(f"Estimated Time Remaining: {remainingTime} m")
        
        start = time.time()
        
        df_1 = df.iloc[i*batch_size: (i*batch_size) + batch_size,:]
        # call the sentiment analizer on our batch
        encoded_data = encode_text(list(df_1['content'].values))
        
        output = model(encoded_data)
        scores = output[0].numpy()
        scores = softmax(scores, axis=1)
        ranking = np.argsort(scores,axis=1)
        ranking = np.flip(ranking, axis=1)
        sentiment = [labels[x[0]] for x in ranking]
        
        # attach the sentiment data to the df we are using
        df_1['sentiment'] = pd.Series(sentiment).values
        
        
        #  merge together into one big df
        if final_df is None:
            final_df = df_1
        else:
            final_df = pd.concat([final_df, df_1],ignore_index=True)
            
        end = time.time()
        
        # let's average out the timer
        executionTime = end - start
        
#         print(end - start)
        
        if(timeEstimates['count']== 0):
            timeEstimates['count'] = 1
            timeEstimates['value'] = executionTime
        else:
            # undo the averaging we did and append this.
            oldTotal = timeEstimates['value'] * timeEstimates['count']
            
            timeEstimates['count'] = timeEstimates['count'] + 1
            timeEstimates['value'] = (oldTotal + executionTime)/timeEstimates['count']
        
            
    if len(df) % batch_size != 0:
        # we have a few remaining items to process
        df_1 = df.iloc[numBatches*batch_size:,:]
        # call the sentiment analizer on our batch
        encoded_data = encode_text(list(df_1['content'].values))
        
        output = model(encoded_data)
        scores = output[0].numpy()
        scores = softmax(scores, axis=1)
        ranking = np.argsort(scores,axis=1)
        ranking = np.flip(ranking, axis=1)
        sentiment = [labels[x[0]] for x in ranking]
        
        # attach the sentiment data to the df we are using
        df_1['sentiment'] = pd.Series(sentiment).values
        final_df = pd.concat([final_df, df_1],ignore_index=True)
#         The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
        
    return final_df

def WorkerJob(onlyfiles, threadNum):
    for app_id in onlyfiles:
        print(f"thread {threadNum}: loading file: {app_id}")
        temp_df = pd.read_pickle(mypath + app_id)
        
        print(f"thread {threadNum}: encoding text for analysis...")
#         temp_df['content'] = temp_df['content'].apply(lambda x: x if x !='None' or x != None else ' None ')
#         display(temp_df['content'].values[:100])
        temp_df['content'] = temp_df['content'].apply(lambda x: 'NA' if x == None or x == 'None' else x)
#         encoded_data = encode_text(list(temp_df['content'].values))
        
        print(f"thread {threadNum}: performing sentiment analysis on: {temp_df['title'].unique()[0]}")
        temp_df = batch_process_df(temp_df, 16)
#      Try using .loc[row_indexer,col_indexer] = value instead   
        
        
        print(f"thread {threadNum}: Done with {app_id}")
        temp_df.to_pickle(mypath + app_id)
        
WorkerJob(onlyfiles, 'only worker')